In [1]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io
import re

In [2]:
paths = ['lstm/data/war-and-peace-1.txt', 
         'lstm/data/war-and-peace-2.txt', 
         'lstm/data/war-and-peace-34.txt']
text_total = ''
for path in paths:
    with io.open(path, encoding="cp1251") as f:
        text = f.read().lower()
    text = text.replace("\n", " ")  # We remove newlines chars for nicer display
    text = re.sub('[^а-я ,.!?-]', '', text) #L Leave only Russian and number and punctuation
    text_total += text
text = text_total
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 2968907
Total chars: 38
Number of sequences: 989623


<ipython-input-2-e18b09359555>:29: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-2-e18b09359555>:30: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [4]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [6]:
epoch = 0
batch_size = 128

In [7]:
epochs = 1
while epoch < epochs:
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print('Epoch', epoch, 'done!')
    epoch += 1

7732/7732 [==============================] - 636s 82ms/step - loss: 1.7950
Epoch 0 done!


In [8]:
print("Generating text after epoch: %d" % 1)
if True:
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

Generating text after epoch: 1
...Diversity: 0.2
...Generating with seed: "ачальствованья.      вернувшись после вт"
...Generated:  орожение и в после собою и простить в перевых и после своего своего приветно и приветения и после своего пристрого под на себе подоманно и приставления к свою после собою сторон и не подомания и своего приставления и своего принять на дела и после под состояли и не подворил на после своего поднял на своего стараться на карагина и после нему на после нему и не видно от нему и после под на себе и пр

...Diversity: 0.5
...Generating with seed: "ачальствованья.      вернувшись после вт"
...Generated:  орой успев на свою на минуту не будет, - как не в перед крыльцы и своего собой, потому, что на нее.      - наташа и на движения и после несколько в предвигавших от голоса, потому когда ни вдруг и как близко и принять в перене и против и даков своего и преказа в петербург. он который он не подумала и подочно проказывал свою положаль и приготовленной положения не в

In [9]:
epochs = 5
while epoch < epochs:
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print('Epoch', epoch, 'done!')
    epoch += 1

7732/7732 [==============================] - 645s 83ms/step - loss: 1.6113
Epoch 1 done!
7732/7732 [==============================] - 654s 85ms/step - loss: 1.5731
Epoch 2 done!
7732/7732 [==============================] - 596s 77ms/step - loss: 1.5513
Epoch 3 done!
7732/7732 [==============================] - 563s 73ms/step - loss: 1.5372
Epoch 4 done!


In [10]:
print("Generating text after epoch: %d" % 5)
if True:
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

Generating text after epoch: 5
...Diversity: 0.2
...Generating with seed: "ть в бурнусе в мечети, так в москве надо"
...Generated:   все полковника и солдата с наполеонами и после подняли с ноги и с своей полковниками и солдата в последней солдат, но не слушали полк было на крестящей самый шагали он с нем состояние с столом, наполеон и солдат расстроить и разговор с том, что он после последний солдат в середине и и с ней и солдат в котором он не спрашивал он не спрашивал с своей рассказывали и после своей собой с которой он не

...Diversity: 0.5
...Generating with seed: "ть в бурнусе в мечети, так в москве надо"
...Generated:   хорошо и в нем возможностью собой в вольситерской глаза, и позиции случает солдат собой о том, что все представлял его с будушь войска направенно спокойное и дарами.  он получил и которое он подошла в том только наполеоных солдата строго и ты не знаюшь на то, что он не счастья и всегда же соня и после так же наполеона дороге по полк на понять от середина кротко 

<ipython-input-5-ded0137f8096>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:   избеначных я остывачивадагы, высшит к тронициклажем вс  люди. о я попновых шал интересоза, не веваства-глакамя. на ней, поднихание моками его, для кого носила от жертго ру. пьер в житавал, любив, анна павловна все казалось, тот блюбочадившими и худтего шапковлья знатательником денисов, строикалось,, и пчеж, нелягла, и было книги солдата свои рейного в жизни и стариковгле. славо собой братиной жиз



In [11]:
epochs = 10
while epoch < epochs:
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print('Epoch', epoch, 'done!')
    epoch += 1

7732/7732 [==============================] - 662s 86ms/step - loss: 1.5264
Epoch 5 done!
7732/7732 [==============================] - 664s 86ms/step - loss: 1.5189
Epoch 6 done!
7732/7732 [==============================] - 670s 87ms/step - loss: 1.5134
Epoch 7 done!
7732/7732 [==============================] - 676s 87ms/step - loss: 1.5088
Epoch 8 done!
7732/7732 [==============================] - 573s 74ms/step - loss: 1.5094
Epoch 9 done!


In [12]:
print("Generating text after epoch: %d" % 10)
if True:
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

Generating text after epoch: 10
...Diversity: 0.2
...Generating with seed: "ражении своем составил для себя роль рук"
...Generated:  ами последние с столом и старать от своей слова.      - вы не видал под того, что после старался и последние в от своей последнее волоса и стараться и старался и последние с себя и последние последние последние подумали и последние с подохох и последний подумал своего последние в своем собой, подвинул князь андрей и от него и подошел к столу, как бы он не понимая с добром и ответить и последние с 

...Diversity: 0.5
...Generating with seed: "ражении своем составил для себя роль рук"


<ipython-input-5-ded0137f8096>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:  ов места далеко, сказал стала наполеона и подвинулся ему потом последние образом стоит не знаю маленькой занимается и подумал он, в дом с неприятельству.  на себе не было собой и удивлялся повернулся в своем своих себе увидал денисова. по последние кавалерии положил подобно от себя.      - и стал самым простом от время потому, что теперь опять был с несколько разрывать и сказал долохов. положение 

...Diversity: 1.0
...Generating with seed: "ражении своем составил для себя роль рук"
...Generated:  и сож-на голу.  чето, анди алексан  нам самые види вседет помощал, анна печен остановился правою кучерии, подвиачно находило и от французский весели, но есть. что немного до будя, но он ар, ?а он ни уторые тех сын готову пишад, окицушение случайности, генерал-расстора и пола с сав весками только менсибря скажет, что я не тец, взглянул князя давал это поспешные, прямо ему ему, гостек-не уторовление

...Diversity: 1.2
...Generating with seed: "ражении своем составил для себя роль

In [5]:
import pickle

In [ ]:
with open('model_wap_10.pickle', 'wb') as file:
    pickle.dump(model.weights, file)

In [6]:
with open('model_wap_15.pickle', 'rb') as file:
    model_weights = pickle.load(file)
model.set_weights(model_weights)

In [7]:
epoch = 15
batch_size = 128

In [8]:
checkpoints = [20, 25, 30, 35, 40, 45, 50, 55, 60]

In [ ]:
for checkpoint in checkpoints:
    # upd model
    while epoch < checkpoint:
        model.fit(x, y, batch_size=batch_size, epochs=1)
        print('Epoch', epoch, 'done!')
        epoch += 1
    # save model
    with open('model_wap_' + str(checkpoint) + '.pickle', 'wb') as file:
        pickle.dump(model.weights, file)
    # gen text
    print("Generating text after epoch: %d" % 10)
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

7732/7732 [==============================] - 716s 92ms/step - loss: 5.1469
Epoch 15 done!
7732/7732 [==============================] - 711s 92ms/step - loss: 4.2278
Epoch 16 done!
7732/7732 [==============================] - 714s 92ms/step - loss: 3.9432
Epoch 17 done!
7732/7732 [==============================] - 700s 91ms/step - loss: 3.6467
Epoch 18 done!
7732/7732 [==============================] - 646s 83ms/step - loss: 3.3820
Epoch 19 done!
Generating text after epoch: 10
...Diversity: 0.2
...Generating with seed: "о неприятеля не останется на моей земле,"


<ipython-input-4-ded0137f8096>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:       о с          пэре    но    п  е     б  о   бао     по  е    н            е  во      о  воне     а      тз    тао     аоо               во  н  нот      о     ва     п             п!           оа  н!            о ноо      - с  о        но      ана       по        во              на т о                во пре пл и с        пое по           е       л        пе о   е       оот по       л сн-     п 

...Diversity: 0.5
...Generating with seed: "о неприятеля не останется на моей земле,"
...Generated:   к. н к не пток  н   пи  ноа  пелал а т.ои а  с п оркалечо бе кеы  и  исс е  ва тмх   онав  сто н  вва оото,  т      и оо всао  вы ссое   осаони син олеиргрл не , ба  а  нливе нне  нот м овеы рде  ов с  к      т  ни бфи, но и  а.    сал и  д             ялано   пооее  пес -у коао н  п  пас ма   о блоис  овет неоа  о ка иикое  м л  бысрантъсва со снавулут   о   птав ви о     к    коес е   т п г   п

...Diversity: 1.0
...Generating with seed: "о неприятеля не останется на моей зе

In [ ]:
epochs = 35
while epoch < epochs:
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print('Epoch', epoch, 'done!')
    epoch += 1

In [ ]:
epochs = 40
while epoch < epochs:
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print('Epoch', epoch, 'done!')
    epoch += 1

In [ ]:
epochs = 45
while epoch < epochs:
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print('Epoch', epoch, 'done!')
    epoch += 1